In [1]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import os
from tqdm import tqdm

## Read sample data

In [2]:
# read sample data
# block
# df_block = pd.read_csv('.\\data\\sample\\blocks.csv', header=None)
# df_block.columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']

# transaction
#df_transaction = pd.read_csv('.\\data\\sample\\transactions.csv', header=None)
#df_transaction.columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']

# input
#df_input = pd.read_csv('.\\data\\sample\\input.csv', header=None)
#df_input.columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address'] # missing a couple of things in the sample (amount)

# output
#df_output = pd.read_csv('.\\data\\sample\\output.csv', header=None)
#df_output.columns = ['transaction id','output_seq','amount_satoshis','address','transaction_hash']

# entities
#df_entities = pd.read_json('.\\data\\sample\\entities.json')


## Full data
### Read data

In [3]:
# set path
path = '.\\gitrepos\\bse_clovrlabs_btc_fraud\\data\\spiderbum-blockchain-upf\\'

#read block files

'''
block_folders = [(path+x, x) for x in os.listdir(path)]

block_columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']
transaction_columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']
input_columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address','amount_satoshis']
output_columns = ['transaction_id','output_seq','amount_satoshis','address','transaction_hash']

df_block = pd.DataFrame(columns=block_columns)
df_transaction = pd.DataFrame(columns=transaction_columns)
df_input = pd.DataFrame(columns=input_columns)
df_output = pd.DataFrame(columns=output_columns)

df_block = pd.concat([pd.read_csv(f[0]+'\\blocks.csv',header=None, names=block_columns) for f in block_folders])
df_transaction = pd.concat([pd.read_csv(f[0]+'\\transactions.csv', header=None, names=transaction_columns) for f in block_folders])
df_input = pd.concat([pd.read_csv(f[0]+'\\input.csv', header=None, names=input_columns) for f in block_folders])
df_output = pd.concat([pd.read_csv(f[0]+'\\output.csv', header=None, names=output_columns) for f in block_folders])

df_block.to_pickle('full_block.pkl')
df_transaction.to_pickle('full_transaction.pkl')
df_input.to_pickle('full_input.pkl')
df_output.to_pickle('full_output.pkl')
'''


"\nblock_folders = [(path+x, x) for x in os.listdir(path)]\n\nblock_columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']\ntransaction_columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']\ninput_columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address','amount_satoshis']\noutput_columns = ['transaction_id','output_seq','amount_satoshis','address','transaction_hash']\n\ndf_block = pd.DataFrame(columns=block_columns)\ndf_transaction = pd.DataFrame(columns=transaction_columns)\ndf_input = pd.DataFrame(columns=input_columns)\ndf_output = pd.DataFrame(columns=output_columns)\n\ndf_block = pd.concat([pd.read_csv(f[0]+'\\blocks.csv',header=None, names=block_columns) for f in block_folders])\ndf_transaction = pd.concat([pd.read_csv(f[0]+'\\transactions.csv', header=None, names=transaction_columns) for f in block_folders])\ndf_input = pd.concat([pd.read_c

In [4]:
# Read block, txn, input and output source pkl files
df_block = pd.read_pickle('.\\data\\full_block.pkl')
df_transaction = pd.read_pickle('.\\data\\full_transaction.pkl')
df_input = pd.read_pickle('.\\data\\full_input.pkl')
df_output = pd.read_pickle('.\\data\\full_output.pkl')

# Read etities
df_entities = pd.read_json('.\\data\\entities.json')
#df_entities.to_csv('entities.csv')

In [5]:
# add block data to txn data
df_transaction = df_transaction.merge(df_block[['block_id','block_hash','block_minted_time']], on=['block_hash'], how='left')

# add input and output data to txn data
merged = df_transaction[:10].merge(df_input, left_on='transaction_hash', right_on='transaction_hash',how='left', suffixes=('','_in'))
merged = merged.merge(df_output, left_on='transaction_hash', right_on='transaction_hash',how='left', suffixes=('','_out'))
merged.rename(columns = {'address':'output_address','amount_satoshis':'amount_satoshis_in'}, inplace=True)

## Test pannel re-creation

In [6]:
# set transaction hash as example
txn = '8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9'

# get selected input and calculate shares and amounts
inputs = merged[merged.transaction_hash == txn][['transaction_hash','coinbase_flag','input_seq','input_address','amount_satoshis_in','block_id','block_minted_time']].drop_duplicates()
inputs['total_amount_in'] = inputs['amount_satoshis_in'].sum()
inputs['amount_share_in'] = inputs['amount_satoshis_in'] / inputs['total_amount_in']

# get selection output and calculate shares and amounts
outputs = merged[merged.transaction_hash == txn][['transaction_hash','output_seq','output_address','amount_satoshis_out']].drop_duplicates()
outputs['total_amount_out'] = outputs['amount_satoshis_out'].sum()
outputs['amount_share_out'] = outputs['amount_satoshis_out'] / outputs['total_amount_out']

# merge input and outputs sample into one
full = outputs.merge(inputs, on=['transaction_hash'], how='left')

# calculate ammounts and fees
full['amount_spread_out'] = full['amount_share_in']*full['amount_satoshis_out']
full['fee'] = full['amount_share_in']*(full['total_amount_in']-full['total_amount_out'])
full['fee_spread'] = full['amount_share_in']*(full['total_amount_in']-full['total_amount_out'])*full['amount_share_out']

full[['transaction_hash','block_id','input_address','output_address','amount_spread_out','fee','fee_spread']]

,transaction_hash,block_id,input_address,output_address,amount_spread_out,fee,fee_spread
0,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2503.64769,1386.37676,3.31484
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,635940.18056,352147.26556,841.98860
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,590236.94681,326839.43114,781.47725
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,101303.22495,56095.92655,134.12608
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,1044603.97555,1386.37676,1383.06192
5,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,265335112.55389,352147.26556,351305.27695
6,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,246266223.62205,326839.43114,326057.95389
7,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,42267029.84851,56095.92655,55961.80047


## Read pannel data

In [7]:
# set table format
pannel_columns = ['txn_hash', 'input_address', 'output_address', 'amount', 'fees', 'block_index', 'block_time']
df_pannel = pd.DataFrame(columns=pannel_columns)

# set path
pannel_path = '.\\gitrepos\\bse_clovrlabs_btc_fraud\\data\\pannel\\'

# read all files, put them together and export to pickle
#pannel_files = [(pannel_path+x, x) for x in os.listdir(pannel_path)]
#df_pannel = pd.concat([pd.read_csv(f[0]) for f in pannel_files])
#df_pannel.to_pickle('full_pannel.pkl')

# read pannel pickle file
df_pannel = pd.read_pickle('.\\data\\full_pannel.pkl')    

In [8]:
# example to compare versus re-created pannel
pannel = df_pannel[df_pannel.txn_hash == txn]
pannel

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2503.64769,1386.37676,453318,2017-02-16 12:05:04 +0000 UTC
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,635940.18056,352147.26556,453318,2017-02-16 12:05:04 +0000 UTC
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,590236.94681,326839.43114,453318,2017-02-16 12:05:04 +0000 UTC
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,101303.22495,56095.92655,453318,2017-02-16 12:05:04 +0000 UTC
5,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,1044603.97555,1386.37676,453318,2017-02-16 12:05:04 +0000 UTC
6,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,265335112.55389,352147.26556,453318,2017-02-16 12:05:04 +0000 UTC
7,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,246266223.62205,326839.43114,453318,2017-02-16 12:05:04 +0000 UTC
8,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,42267029.84851,56095.92655,453318,2017-02-16 12:05:04 +0000 UTC


## Clovr Data
### Read data

In [9]:
# Read block, txn, input and output source pkl files
df_block = pd.read_pickle('.\\data\\full_block.pkl')
df_transaction = pd.read_pickle('.\\data\\full_transaction.pkl')
df_input = pd.read_pickle('.\\data\\full_input.pkl')
df_output = pd.read_pickle('.\\data\\full_output.pkl')

# Read entities
df_entities = pd.read_json('.\\data\\entities.json')

### Get unique address

In [10]:
# get unique addresses from input and output and generate csv with list
all_address = df_input['input_address'].append(df_output['address'], ignore_index=True)
all_address = all_address.drop_duplicates().reset_index()

all_address = pd.DataFrame(all_address.iloc[:,1])
all_address.columns = ['address']
all_address.to_csv('.\\data\\unique_address.csv', header=None)
print(len(all_address))

4709734


### Get matches from labels.csv

In [11]:
# find addresses that are labelled
labels_reader = pd.read_csv('.\\data\labels.csv', 
                          chunksize = 1000000,
                          names=['address','id'])

os.remove('.\\data\\clovr_address_matching_labels.csv')

for chunk in labels_reader:
    chunk = chunk[chunk['address'].isin(all_address['address'])]
    chunk.to_csv('.\\data\\clovr_address_matching_labels.csv', mode='a', index=False, header=None)

### Get unique addressess matching labels

In [12]:
clovr_address_matching_labels = pd.read_csv('.\\data\\clovr_address_matching_labels.csv', header = None)
clovr_address_matching_labels.columns = ['address','name']

In [13]:
unique_clovr_address_matching_labels = clovr_address_matching_labels['address'].unique()
print(len(clovr_address_matching_labels))
print(len(unique_clovr_address_matching_labels))

434664
434664


### Get unique names matching labels

In [14]:
unique_clovr_name_matching_labels = clovr_address_matching_labels['name'].unique()
print(len(clovr_address_matching_labels))
print(len(unique_clovr_name_matching_labels))

434664
61


### Get matches from labels extended

In [15]:
# find addresses that are labelled
labels_extended_reader = pd.read_csv('.\\data\labels_extended_original.csv', 
                          chunksize = 1000000,
                          names=['address','id', 'entity','category','user_score'])

os.remove('.\\data\\clovr_address_matching_labelsext.csv')

for chunk in labels_extended_reader:
    chunk = chunk[chunk['address'].isin(all_address['address'])]
    chunk.to_csv('.\\data\\clovr_address_matching_labelsext.csv', mode='a', index=False, header=None)

In [16]:
clovr_address_matching_labelsext = pd.read_csv('.\\data\\clovr_address_matching_labelsext.csv', header=None, names=['address','id','name','category','user_score'])
print(len(clovr_address_matching_labelsext))

53895


## Elliptic Data
### Read data

In [17]:
# read data
ell_transaction = pd.read_csv('.\\data\\elliptic_dataset\\transactions.csv')
ell_input = pd.read_csv('.\\data\\elliptic_dataset\\inputs.csv')
ell_output = pd.read_csv('.\\data\\elliptic_dataset\\outputs.csv')
ell_deanon = pd.read_csv('.\\data\\elliptic_dataset\\deanonymised_elliptic.csv')

### Get unique addresses

In [18]:
# convert from str to list and create a list of unique input/output addresses
import ast
ell_input['addresses'] = [ast.literal_eval(n) for n in ell_input['addresses']]
ell_address_in = pd.DataFrame(ell_input['addresses']).explode('addresses', ignore_index = True)

ell_output['addresses'] = [ast.literal_eval(n) for n in ell_output['addresses']]
ell_address_out = pd.DataFrame(ell_output['addresses']).explode('addresses', ignore_index = True)

ell_all_address = ell_address_in.append(ell_address_out, ignore_index=True)
ell_all_address = ell_all_address.drop_duplicates().reset_index()
ell_all_address.drop(columns=['index'], inplace=True)
print(len(ell_all_address))

823444


### Get matches from labels.csv

In [19]:
# find addresses that are labelled
labels_reader = pd.read_csv('.\\data\labels.csv', 
                          chunksize = 1000000,
                          names=['address','id'])

os.remove('.\\data\\elliptic_address_matching_labels.csv')

for chunk in labels_reader:
    chunk = chunk[chunk['address'].isin(ell_all_address['addresses'])]
    chunk.to_csv('.\\data\\elliptic_address_matching_labels.csv', mode='a', index=False, header=None)

### Get unique addresses matching labels

In [20]:
elliptic_address_matching_labels = pd.read_csv('.\\data\\elliptic_address_matching_labels.csv', header = None)
elliptic_address_matching_labels.columns= ['address','name']

In [21]:
unique_elliptic_address_matching_labels = elliptic_address_matching_labels['address'].unique()
print(len(elliptic_address_matching_labels))
print(len(unique_elliptic_address_matching_labels))

165894
165894


### Get unique names matching labels

In [22]:
unique_elliptic_name_matching_labels = elliptic_address_matching_labels['name'].unique()
print(len(elliptic_address_matching_labels))
print(len(unique_elliptic_name_matching_labels))

165894
59


### Get matches from labels extended

In [23]:
# find addresses that are labelled
labels_extended_reader = pd.read_csv('.\\data\labels_extended_original.csv', 
                          chunksize = 1000000,
                          names=['address','id', 'entity','category','user_score'])

#os.remove('.\\data\\elliptic_address_matching_labelsext.csv')

for chunk in labels_extended_reader:
    chunk = chunk[chunk['address'].isin(ell_all_address['addresses'])]
    chunk.to_csv('.\\data\\elliptic_address_matching_labelsext.csv', mode='a', index=False, header=None)

In [24]:
elliptic_address_matching_labelsext = pd.read_csv('.\\data\\elliptic_address_matching_labelsext.csv', header=None, names=['address','id','name','category','user_score'])
print(len(elliptic_address_matching_labelsext))

61896


## Labels comparison to Entities
### Analyze extended labels

In [25]:
# read original labels data
labels_extended = pd.read_csv('.\\data\\labels_extended_original.csv', header=None, names=['address','index','name','category','user_score'])

# eliminate duplicate rows
unique_labels_extended = labels_extended.drop_duplicates()

# export to csv
unique_labels_extended[['address','index','name','category','user_score']].to_csv('c:\\temp\\unique_labels_extended.csv')

In [26]:
print(len(labels_extended))
print(len(unique_labels_extended))

14565812
14565114


### Analyze labels

In [28]:
labels = pd.read_pickle('.\\data\\labels.pkl')

In [29]:
unique_names_from_labels = labels['entity'].drop_duplicates()
len(unique_names_from_labels)

839

In [30]:
unique_names_from_labels = pd.DataFrame(unique_names_from_labels)
labels_not_entities = unique_names_from_labels.merge(df_entities, left_on='entity', right_on='name', how='left')
entities_not_labels = unique_names_from_labels.merge(df_entities, left_on='entity', right_on='name', how='right')

len(labels_not_entities[labels_not_entities.name.isna()])

78

In [31]:
len(entities_not_labels[entities_not_labels.entity.isna()])

922

In [35]:
labels_not_entities[labels_not_entities.name.isna()].to_csv('.\\data\\labels_not_entities.csv')